# Retrieving all related ICD9 codes

- HCUP Procedure Class 4: Major therapeutic procedure
- CCS classification

In [72]:
import pandas as pd
import sqlalchemy as sal

engine = sal.create_engine('mssql+pyodbc://@SIA09-ICPR01.SIHMIS.SI/SiIMC_MGHT?trusted_connection=yes&driver=SQL+Server')
conn = engine.connect()


### HCUP Procedure Class

In [73]:
pc = pd.read_csv('..\data\proc_code\pc.csv') # read hcup procedure class file
pc.columns = ['icd9cm_code', 'icd9cm_desc', 'pc_class']

In [74]:
filter_class4 = pc[pc.iloc[:, 2] == 4] # filter only class 4: 
filter_class4.loc[:, 'icd9cm_code'] = filter_class4['icd9cm_code'].str.replace("'", "") # cleaning up icd code
filter_class4['icd9cm_code_with_dot'] = filter_class4['icd9cm_code'].str[:2] + '.' + filter_class4['icd9cm_code'].str[2:] # add dot to icd9 code
filter_class4['icd9cm_code_with_dot'] = filter_class4['icd9cm_code_with_dot'].str.strip() # remove space
filter_class4 = filter_class4.drop('pc_class', axis=1)

C:\Users\krittaphas.chi\AppData\Local\Temp\ipykernel_10144\530191331.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_class4['icd9cm_code_with_dot'] = filter_class4['icd9cm_code'].str[:2] + '.' + filter_class4['icd9cm_code'].str[2:] # add dot to icd9 code
C:\Users\krittaphas.chi\AppData\Local\Temp\ipykernel_10144\530191331.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_class4['icd9cm_code_with_dot'] = filter_class4['icd9cm_code_with_dot'].str.strip() # remove space


In [75]:
with open('..\sql\procedure_concept_map.sql') as file:
    sql_command = file.read()
    print(sql_command)
    vocab_concept = pd.read_sql(sql_command, conn)
    file.close()
vocab_concept.head()

SELECT concept_id, concept_name, concept_code
FROM [SiIMC_MGHT].[vocab].[concept]
WHERE domain_id = 'Procedure'
AND vocabulary_id = 'ICD9Proc'
ORDER BY concept_code



,concept_id,concept_name,concept_code
0,2000002,"Procedures and interventions, Not Elsewhere Cl...",00
1,2000003,Therapeutic ultrasound,00.0
2,2000006,Therapeutic ultrasound of vessels of head and ...,00.01
3,2000007,Therapeutic ultrasound of heart,00.02
4,2000008,Therapeutic ultrasound of peripheral vascular ...,00.03


In [76]:
merge_df = filter_class4.merge(vocab_concept, left_on='icd9cm_code_with_dot', right_on='concept_code', how='left')

In [77]:
merge_df = merge_df[['icd9cm_code_with_dot', 'concept_code', 'icd9cm_desc', 'concept_name', 'concept_id']]
merge_df.columns = ['icd9_hcup', 'icd9_omop', 'hcup_desc', 'omop_desc', 'concept_id']

### CCS classification

From this classification we only use CCS level 2 for classification

In [78]:
ccs = pd.read_csv('..\data\proc_code\ccs.csv')

ccs.columns = ['icd9', 'ccs1', 'ccs1_label', 'ccs2', 'ccs2_label', 'ccs3', 'ccs3_label']

ccs["icd9"] = ccs["icd9"].str.replace("'", "")
ccs["icd9"] = ccs["icd9"].str[:2] + "." + ccs["icd9"].str[2:]

ccs['ccs1'] = ccs['ccs1'].str.replace("'", "")
ccs['ccs2'] = ccs['ccs2'].str.replace("'", "")
ccs['ccs3'] = ccs['ccs3'].str.replace("'", "")

# only use CCS level 2 Classification
ccs = ccs.drop(['ccs1', 'ccs1_label', 'ccs3', 'ccs3_label'], axis=1)
ccs['icd9'] = ccs['icd9'].str.strip()

ccs.head()

,icd9,ccs2,ccs2_label
0,01.21,1.1,Incision and excision of CNS [1.]
1,01.22,1.1,Incision and excision of CNS [1.]
2,01.23,1.1,Incision and excision of CNS [1.]
3,01.24,1.1,Incision and excision of CNS [1.]
4,01.25,1.1,Incision and excision of CNS [1.]


In [79]:
merge_df = merge_df.merge(ccs, left_on='icd9_omop', right_on='icd9', how='left')
merge_df.head()

,icd9_hcup,icd9_omop,hcup_desc,omop_desc,concept_id,icd9,ccs2,ccs2_label
0,00.50,00.50,IMPLA RESYNCHR PACEMAKER W/0 (Begin 2002),Implantation of cardiac resynchronization pace...,2000048.0,00.50,7.6,Insertion; revision; replacement; removal of c...
1,00.51,00.51,IMPLA RESYNCHRONIZATION DEFI (Begin 2002),Implantation of cardiac resynchronization defi...,2000049.0,00.51,7.6,Insertion; revision; replacement; removal of c...
2,00.52,00.52,IMPL/REPL TRANSVENOUS LEAD L (Begin 2002),Implantation or replacement of transvenous lea...,2000050.0,00.52,7.6,Insertion; revision; replacement; removal of c...
3,00.53,00.53,IMPL/REPL PACEMAKER PLSE GE (Begin 2002),Implantation or replacement of cardiac resynch...,2000051.0,00.53,7.6,Insertion; revision; replacement; removal of c...
4,00.54,00.54,IMPL/REPL DEFIBRIL GENERATOR (Begin 2002),Implantation or replacement of cardiac resynch...,2000052.0,00.54,7.6,Insertion; revision; replacement; removal of c...


In [80]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2373 entries, 0 to 2372
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   icd9_hcup   2373 non-null   object 
 1   icd9_omop   2357 non-null   object 
 2   hcup_desc   2373 non-null   object 
 3   omop_desc   2357 non-null   object 
 4   concept_id  2357 non-null   float64
 5   icd9        2357 non-null   object 
 6   ccs2        2357 non-null   object 
 7   ccs2_label  2357 non-null   object 
dtypes: float64(1), object(7)
memory usage: 148.4+ KB


##### Check Nan
There are 16 Nan rows

In [81]:
nan_df = merge_df[merge_df.isnull().any(axis=1)]
# nan_df.to_csv('..\data\proc_code\proc_omop_nan.csv', index=False)

In [82]:
nan_df

,icd9_hcup,icd9_omop,hcup_desc,omop_desc,concept_id,icd9,ccs2,ccs2_label
350,13.61,NaN,DISC PRIME MEM CATARACT (Begin 1980 End 1991),NaN,NaN,NaN,NaN,NaN
351,13.62,NaN,EXC PRIME MEM CATARACT (Begin 1980 End 1991),NaN,NaN,NaN,NaN,NaN
352,13.63,NaN,MECH FRAG MEM CATARACT (Begin 1980 End 1991),NaN,NaN,NaN,NaN,NaN
753,36.00,NaN,REM OBSTR NOS (Begin 1986 End 1991),NaN,NaN,NaN,NaN,NaN
1024,43.2,NaN,PERM GASTROSTOMY(Begin 1980 End 1989),NaN,NaN,NaN,NaN,NaN
1105,46.12,NaN,PERM MAGNETIC COLOSTOMY (End 1992),NaN,NaN,NaN,NaN,NaN
1167,48.66,NaN,HARTMANN RESECTION RECTUM (Begin 1980 End 1988),NaN,NaN,NaN,NaN,NaN
1278,52.91,NaN,ERCP (Begin 1980 End 1989),NaN,NaN,NaN,NaN,NaN
1446,59.01,NaN,URETEROLYS FOR FIBROSIS,NaN,NaN,NaN,NaN,NaN
1636,69.11,NaN,REMOVE INTRALIG ECT PREG (End 1992),NaN,NaN,NaN,NaN,NaN


These Nan codes are discontinued codes.

#### Drop nan row from merge_df

In [83]:
merge_df = merge_df.dropna()

In [84]:
merge_df = merge_df[['icd9_hcup', 'icd9_omop', 'icd9', 'ccs2', 'ccs2_label', 'hcup_desc', 'omop_desc', 'concept_id']]
merge_df.columns = ['icd9_hcup', 'icd9_omop', 'icd9_ccs', 'ccs2', 'ccs2_desc', 'hcup_desc', 'omop_desc', 'concept_id']
merge_df['concept_id'] = merge_df['concept_id'].astype(int)

In [86]:
# merge_df.to_csv('..\data\proc_code\proc_hcup_omop.csv', index=False)

In [88]:
pd.set_option('display.max_rows', None)
merge_df

,icd9_hcup,icd9_omop,icd9_ccs,ccs2,ccs2_desc,hcup_desc,omop_desc,concept_id
0,00.50,00.50,00.50,7.6,Insertion; revision; replacement; removal of c...,IMPLA RESYNCHR PACEMAKER W/0 (Begin 2002),Implantation of cardiac resynchronization pace...,2000048
1,00.51,00.51,00.51,7.6,Insertion; revision; replacement; removal of c...,IMPLA RESYNCHRONIZATION DEFI (Begin 2002),Implantation of cardiac resynchronization defi...,2000049
2,00.52,00.52,00.52,7.6,Insertion; revision; replacement; removal of c...,IMPL/REPL TRANSVENOUS LEAD L (Begin 2002),Implantation or replacement of transvenous lea...,2000050
3,00.53,00.53,00.53,7.6,Insertion; revision; replacement; removal of c...,IMPL/REPL PACEMAKER PLSE GE (Begin 2002),Implantation or replacement of cardiac resynch...,2000051
4,00.54,00.54,00.54,7.6,Insertion; revision; replacement; removal of c...,IMPL/REPL DEFIBRIL GENERATOR (Begin 2002),Implantation or replacement of cardiac resynch...,2000052
5,00.56,00.56,00.56,7.6,Insertion; revision; replacement; removal of c...,INS/REP IMPL SENSOR LEAD (Begin 2006),Insertion or replacement of implantable pressu...,2000054
6,00.57,00.57,00.57,7.6,Insertion; revision; replacement; removal of c...,IMP/REP SUBCUE CARD DEV (Begin 2006),Implantation or replacement of subcutaneous de...,2000055
7,00.61,00.61,00.61,7.17,Other OR procedures on vessels of head and nec...,PERC ANGIO PRECEREB VESS (Begin 2004),Percutaneous angioplasty of extracranial vesse...,2000059
8,00.62,00.62,00.62,7.17,Other OR procedures on vessels of head and nec...,PERC ANGIO INTRACRAN VES (Begin 2004),Percutaneous angioplasty of intracranial vesse...,2000060
9,00.66,00.66,00.66,7.3,Percutaneous transluminal coronary angioplasty...,PTCA OR CORONARY ATHER (Begin 2005),Percutaneous transluminal coronary angioplasty...,2000064
